## Scenario 2: A cross-functional team with one data scientist working on an ML model


MLflow setup:
- tracking server: yes, local server
- backend store: sqlite database
- artifacts store: local filesystem

The experiments can be explored locally by accessing the local tracking server.

To run this example you need to launch the mlflow server locally by running the following command in your terminal:

`mlflow server --backend-store-uri sqlite:///backend.db`

In [2]:
import mlflow


mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [3]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5000'


In [4]:
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1723484369420, experiment_id='0', last_update_time=1723484369420, lifecycle_stage='active', name='Default', tags={}>]

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2024/08/12 20:43:51 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.
2024/08/12 20:43:58 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/08/12 20:43:58 INFO mlflow.tracking._tracking_service.client: 🏃 View run handsome-bear-539 at: http://127.0.0.1:5000/#/experiments/1/runs/080d994b6cd04c73900328c57098cad4.
2024/08/12 20:43:58 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1.


default artifacts URI: 'mlflow-artifacts:/1/080d994b6cd04c73900328c57098cad4/artifacts'


In [6]:
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1723484631503, experiment_id='1', last_update_time=1723484631503, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1723484369420, experiment_id='0', last_update_time=1723484369420, lifecycle_stage='active', name='Default', tags={}>]

### Interacting with the model registry

In [7]:
from mlflow.tracking import MlflowClient


client = MlflowClient("http://127.0.0.1:5000")

In [9]:
client.search_registered_models()

[]

In [17]:
# Correct usage of search_runs
run_id = client.search_runs(experiment_ids=['1'])[0].info.run_id

# Register the model
mlflow.register_model(
    model_uri=f"runs:/{run_id}/model",
    name='iris-classifier'
)


Successfully registered model 'iris-classifier'.
2024/08/12 20:57:46 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1723485466510, current_stage='None', description='', last_updated_timestamp=1723485466510, name='iris-classifier', run_id='080d994b6cd04c73900328c57098cad4', run_link='', source='mlflow-artifacts:/1/080d994b6cd04c73900328c57098cad4/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>